In [1]:
!pip install vllm
!pip install pygoogletranslation
!pip install googletrans==4.0.0-rc1

In [2]:
from vllm import LLM, SamplingParams

llm = LLM(model="mistralai/Mistral-7B-Instruct-v0.1",trust_remote_code=True,dtype="auto")

INFO 12-16 20:30:41 llm_engine.py:73] Initializing an LLM engine with config: model='mistralai/Mistral-7B-Instruct-v0.1', tokenizer='mistralai/Mistral-7B-Instruct-v0.1', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, seed=0)
INFO 12-16 20:30:52 llm_engine.py:222] # GPU blocks: 8921, # CPU blocks: 2048


**Structure Output with One shot Learning**

**Template for Mistral-7B-Instruct**

`<s>[INST] Instruction [/INST] Model answer</s>[INST] Follow-up instruction [/INST]`

In [3]:
from pygoogletranslation import Translator
translator = Translator()

In [100]:
#prompts = "နေ့တစ်နေ့ကို အကျိုးရှိရှိ ဘယ်လိုစတင်မလဲ ?"
prompts = "မြန်မာနိုင်ငံမှာ စိတ်ဝင်စားစရာအကောင်းဆုံးနေရာတွေက ဘာတွေလဲ ?"
#prompts =  "ပရိုဂရမ်းမင်းကို ဘယ်လိုစလေ့လာရမလဲ ?"


prompt = translator.translate(f'{prompts}', dest='en')
query = prompt.text


In [101]:
Instruction = """
You are a helpful AI assistant. Your task is to generate a JSON format based on the given information.
Think step by step.

Do you have a morning ritual?

would be converted to:
"""

sample_answer = """
{
- Wake up early (6:30 am) and get out of bed.
- Drink a glass of water to hydrate myself.
- Exercise for 30 minutes (usually yoga or running).
- Take a shower to start the day fresh.
}
"""

user_question = f"""
{query} :
"""

prompt_template = f"<s>[INST] {Instruction} [/INST] {sample_answer} </s>[INST] {user_question} [/INST]"

In [102]:
sampling_params = SamplingParams(temperature=0.5, top_p=0.95,max_tokens=300)
outputs = llm.generate(prompt_template, sampling_params)

result = " "
for output in outputs:
    generated_text = output.outputs[0].text
    translator = Translator()
    result = translator.translate(f'{generated_text}', dest='my')



Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.28s/it]


In [103]:
print(f"Prompt: {prompts!r}")
print(" ")
print(f"Answer: {result.text}")

Prompt: 'မြန်မာနိုင်ငံမှာ စိတ်ဝင်စားစရာအကောင်းဆုံးနေရာတွေက ဘာတွေလဲ ?'
 
Answer: {
- ပုဂံ - ထောင်ပေါင်းများစွာသောဗုဒ္ဓဘာသာဘုန်းတော်ကြီးကျောင်းများနှင့်ဘုရားတို့နှင့်အတူယူနက်စကိုကမ္ဘာ့အမွေအနှစ်နေရာ။
- ရန်ကုန် - မြန်မာနိုင်ငံ၏အကြီးဆုံးမြို့ဖြစ်သောရွှေတိဂုံဘုရားနှင့်ရွှေတိဂုံစေတီဇာအပါအ 0 င်ကြွယ်ဝသောသမိုင်းနှင့်ယဉ်ကျေးမှုတစ်ခုဖြင့်မြန်မာနိုင်ငံ၏အကြီးဆုံးမြို့။
- မန္တလေး - မြန်မာနိုင်ငံအလယ်ပိုင်းရှိမြို့တစ်မြို့သည်လှပသောဘုရားကျောင်းများ,
- အင်းလေးကန် - တောင်တန်းများနှင့်အတူတစ် ဦး ရင်သပ်ရှုမောဖွယ်ရေကန်များနှင့်အတူတစ် ဦး နှင့်အတူတောင်တန်းများနှင့်တစ် ဦး တည်းသောဘဝလမ်းစဉ်နှင့်ရိုးရာလှေတည်ဆောက်ရေးနည်းစနစ်များအတွက်လူသိများသောတစ် ဦး နှင့်အတူတစ် ဦး နှင့်အတူအိမ်များနှင့်အိမ်များနှင့်အတူတစ် ဦး နှင့်အတူဝိုင်းရံ။
- နေပြည်တော် - ခေတ်သစ်ဗိသုကာ, ပြတိုက်များနှင့်ယဉ်ကျေးမှုဆိုင်ရာအထင်ကရနေရာများရှိသောမြန်မာနိုင်ငံ၏မြို့တော်မြို့တော်ဖြစ်သည်။
}
